In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
from PIL import Image
import os

In [ ]:
data_path="/kaggle/input/brain-tumor-classification-mri/Training"

In [ ]:
ext_data=[]
for images in os.listdir(data_path):
    image=os.path.join(data_path,images)
    ext_data.append(image)
    

In [ ]:
var1="/kaggle/input/brain-tumor-classification-mri/Training/glioma_tumor"

In [ ]:
var1_data=[]
for images in os.listdir(var1):
    image=os.path.join(var1,images)
    image=Image.open(image)
    var1_data.append(image)

In [ ]:
# var1_data.isnull()

In [ ]:
var1_data

In [ ]:
# for id,image in enumerate(var1_data):
#     image1=image.show(image)
# #     print(new_image)

In [ ]:
image

In [ ]:
# cv2.imshow(var1_data)


In [ ]:
# cv2.imshow("Image",var1_data)

In [ ]:
# var1_data

In [ ]:
# ext_data

In [ ]:
# x=cv2.imread("/kaggle/input/brain-tumor-classification-mri/Training/glioma_tumor")

In [ ]:
# cv2.imshow("Imgae",x)

In [ ]:
from PIL import Image
import os

# Directory containing the images
dataset_dir = "/kaggle/input/brain-tumor-classification-mri/Training/glioma_tumor"

# List to store image data
images = []

# Iterate over files in the directory
for filename in os.listdir(dataset_dir):
    if filename.endswith(".jpg") or filename.endswith(".png"):  # Filter images
        # Read the image
        image_path = os.path.join(dataset_dir, filename)
        image = cv2.imread(image_path)
        if image is not None:
            images.append(image)
#         images.append(image)
        else:
            print("unable to load")
# images.show()

# # # Display all images
# for idx, image in enumerate(images):
#     image.show()
#     input("Press Enter to display the next image or Ctrl+C to exit...")


In [ ]:
# cv2.imshow("Image",images)

In [ ]:
# import cv2
# import os
# import random

# # Directory containing the images
# dataset_dir = "/kaggle/input/brain-tumor-classification-mri/Training/glioma_tumor"
# # List all image files in the directory
# image_files = [os.path.join(dataset_dir, file) for file in os.listdir(dataset_dir) if file.endswith(('.png', '.jpg', '.jpeg'))]

# # Number of images to display
# num_images_to_display = 10

# # Randomly select images to display
# images_to_display = random.sample(image_files, min(num_images_to_display, len(image_files)))

# # Display each image
# for image_file in images_to_display:
#     # Read the image
#     image = cv2.imread(image_file)
#     if image is not None:
#         # Display the image
#         cv2.imshow("Image", image)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()
#     else:
#         print(f"Unable to read image: {image_file}")


In [ ]:
import cv2
import os
import random
import matplotlib.pyplot as plt

# Directory containing the images
dataset_dir = "/kaggle/input/brain-tumor-classification-mri/Training/glioma_tumor"

# List all image files in the directory
image_files = [os.path.join(dataset_dir, file) for file in os.listdir(dataset_dir) if file.endswith(('.png', '.jpg', '.jpeg'))]

# Number of images to display
num_images_to_display = 10

# Randomly select images to display
images_to_display = random.sample(image_files, min(num_images_to_display, len(image_files)))

# Display each image
for image_file in images_to_display:
    # Read the image
    image = cv2.imread(image_file)
    if image is not None:
        # Convert BGR to RGB (matplotlib uses RGB)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Display the image
        plt.imshow(image)
#         plt.axis('off')  # Hide axis
        plt.show()
    else:
        print(f"Unable to read image: {image_file}")


In [89]:
from PIL import Image

# Path to the PNG file
file_path = var1_data  # Replace this with the path to your PNG file

# Read the PNG file
try:
    image = Image.open(file_path)
    # Optionally, you can perform further processing or display the image here
    image.show()  # Opens the image using the default image viewer
except IOError:
    print("Unable to open or read the PNG file")


AttributeError: 'list' object has no attribute 'read'

In [ ]:
xdg-open /tmp/tmppvthta44.PNG

In [90]:
import cv2
import numpy as np

# Define the desired dimensions for resizing
desired_width = 100
desired_height = 100

# List to store resized image arrays
image_arrays = []

# Loop through all image file paths
for image_path in data:
    # Read the image using cv2
    image = cv2.imread(image_path)
    
    # Resize the image to the desired dimensions
    resized_image = cv2.resize(image, (desired_width, desired_height))
    
    # Append the resized image array to the list
    image_arrays.append(resized_image)

# Convert the list of image arrays into a NumPy array
image_array = np.array(image_arrays)

# Now, 'image_array' contains the resized arrays of all images with consistent dimensions


error: OpenCV(4.9.0) /io/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [ ]:
xdg-open --debug /tmp/tmppvthta44.PNG


In [ ]:
import cv2
import os
import random
import matplotlib.pyplot as plt

In [ ]:
data_pathz="/kaggle/input/brain-tumor-classification-mri/Training/glioma_tumor"

In [91]:
# def extract_data(data_pathz):
new_file_data=[]
data_file_img=[os.path.join(data_pathz,image) for image in os.listdir(data_pathz)]
# for image in os.listdir(data_pathz):
#     new_file = os.path.join(data_pathz,image)
new_file_random=random.sample(data_file_img, min(10,len(new_file)))
for random_new_img in new_file_random:
#         if random_new_img is not None:
    new_file1=cv2.imread(random_new_img)
    if new_file1 is not None:
        new_file_data.append(new_file1)
#         plt.imshow(new_file1)
#         plt.show()
    else:
        print("not found")
#             np.array(new_file_data)
#     if new_file_data is not None:
#         plt.imshow(new_file_data[0])
#         plt.show()
#     return new_file_data

In [ ]:
import matplotlib.pyplot as plt

# Assuming new_file_data is your list of images
# new_file_data = [...]

num_images = len(new_file_data)

# Calculate number of rows and columns for subplots
num_cols = 3
num_rows = (num_images + num_cols - 1) // num_cols

# Create subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(12, 8))

# Plot each image
for i, ax in enumerate(axes.flat):
    if i < num_images:
        row = i // num_cols
        col = i % num_cols
        ax.imshow(new_file_data[i])
        ax.set_title(f"Image {i+1}")
        ax.axis('off')  # Hide axis
    else:
        ax.axis('off')  # Hide empty subplot

# Adjust layout to prevent overlap
plt.tight_layout()

# Show plot
plt.show()


In [ ]:
plt.subplot(3,3,9)
plt.plot(new_file_data)

In [ ]:
new_file_data
# plt.imshow(new_file_data)
# plt.show()

In [ ]:
# def extract_data(data_pathz):
new_file_data=[]
data_file_img=[os.path.join(data_pathz,image) for image in os.listdir(data_pathz)]
# for image in os.listdir(data_pathz):
#     new_file = os.path.join(data_pathz,image)
new_file_random=random.sample(data_file_img, min(10,len(new_file)))
for random_new_img in new_file_random:
#         if random_new_img is not None:
    new_file1=cv2.imread(random_new_img)
    if new_file1 is not None:
#         new_file_data.append(new_file1)
        plt.imshow(new_file1)
        plt.show()
    else:
        print("not found")
#             np.array(new_file_data)
#     if new_file_data is not None:
#         plt.imshow(new_file_data[0])
#         plt.show()
#     return new_file_data

In [ ]:
import cv2
import os
import random
import matplotlib.pyplot as plt

# Directory containing the images
dataset_dir = "/kaggle/input/brain-tumor-classification-mri/Training/glioma_tumor"

# List all image files in the directory
image_files = [os.path.join(dataset_dir, file) for file in os.listdir(dataset_dir) if file.endswith(('.png', '.jpg', '.jpeg'))]

# Number of images to display
num_images_to_display = 10

# Randomly select images to display
images_to_display = random.sample(image_files, min(num_images_to_display, len(image_files)))

# Display each image
for image_file in images_to_display:
    # Read the image
    image = cv2.imread(image_file)
    if image is not None:
        # Convert BGR to RGB (matplotlib uses RGB)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Display the image
        plt.imshow(image)
#         plt.axis('off')  # Hide axis
        plt.show()
    else:
        print(f"Unable to read image: {image_file}")


In [ ]:
dir_train_path="/kaggle/input/brain-tumor-classification-mri/Training"

In [ ]:
# new_img10=[]
# for images in os.listdir(dir_train_path):
#     ima=os.path.join(dir_train_path,images)
#     if ima is not None:
#         ima=cv2.imread(ima)
#         new_img10.append(ima)
#     for image in images:
#         new_img1=os.path.join(ima,image)
#         if new_img1 is not None:
#             new_img2=cv2.imread(new_img1)
#             new_img10.append(new_img2)
       

In [ ]:
# new_img

In [ ]:
data="/kaggle/input/brain-tumor-classification-mri/Training/meningioma_tumor"
data_no_tumor="/kaggle/input/brain-tumor-classification-mri/Training/no_tumor"
data_pituitary_tumor="/kaggle/input/brain-tumor-classification-mri/Training/pituitary_tumor"

In [ ]:
# import random
# random_image=random.sample(data,(10,len(data)))
data

In [ ]:
# data=pd.DataFrame(x,columns=["Glioma_tumor"])
glioma_tumor=extract_data(data_pathz)
meningioma_tumor=extract_data(data)
no_tumor=extract_data(data_no_tumor)
pituitary_tumor=extract_data(data_pituitary_tumor)

In [ ]:
print(meningioma_tumor)
print(len(glioma_tumor))
print(len(no_tumor))
print(len(pituitary_tumor))

In [ ]:
df=pd.DataFrame(meningioma_tumor)
df.columns = [f'Pixel_{i}' for i in range(df.shape[1])]

In [ ]:
import dopamine
from dopamine.agents.dqn import dqn_agent
from dopamine.atari import run_experiment
from dopamine.colab import utils as colab_utils
from absl import flags

# Define flags for Dopamine
BASE_PATH = '/tmp/colab_dope_run'
LOG_PATH = os.path.join(BASE_PATH, 'basic_agent')

# Set up flags to configure Dopamine
LOG_PATH = os.path.join('/tmp/colab_dope_run', 'basic_agent')
# Number of training steps
flags.DEFINE_integer('num_iterations', 200, 'Number of training iterations.')
# Path to the level script
flags.DEFINE_string('game', 'CartPole-v0', 'Name of the Atari game to run.')

# Flags to pass to the agent
flags.DEFINE_boolean('verbose_logging', False, 'Whether to log progress verbose.')
flags.DEFINE_boolean('notify_training_progress', True, 'Whether to log training progress.')
flags.DEFINE_string('agent_name', 'dqn', 'Name of the agent to run.')

FLAGS = flags.FLAGS


def create_dqn_agent(sess, environment):
    """Creates a DQN agent."""
    return dqn_agent.DQNAgent(sess, num_actions=environment.action_space.n)


def main():
    # Set up logging
    colab_utils.mount_google_drive()
    experiment_logger = colab_utils.create_default_logger(LOG_PATH)

    # Create an environment
    environment = colab_utils.create_environment(FLAGS.game)

    # Create a TensorFlow session
    sess = colab_utils.create_session()

    # Create the DQN agent
    agent = create_dqn_agent(sess, environment)

    # Initialize the agent
    agent.initialize()

    # Train the agent
    run_experiment.run_experiment(
        agent=agent,
        environment=environment,
        num_iterations=FLAGS.num_iterations,
        training_steps=FLAGS.training_steps,
        verbose_logging=FLAGS.verbose_logging,
        notify_training_progress=FLAGS.notify_training_progress,
        log_path=LOG_PATH


In [ ]:
!pip install stable_baselines

In [ ]:
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy

# Create the CartPole environment
env = gym.make('CartPole-v1')

# Create and train the PPO agent
model = PPO('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=10000)

# Save the trained model
model.save("ppo_cartpole")

# Load the trained model
loaded_model = PPO.load("ppo_cartpole")

# Evaluate the trained model
mean_reward, _ = evaluate_policy(loaded_model, env, n_eval_episodes=10)
print("Mean reward:", mean_reward)


In [ ]:
import gym
from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy

# Create the CartPole environment
env = gym.make('CartPole-v1')

# Define and train the DQN agent
model = DQN("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=10000)

# Save the trained model
model.save("dqn_cartpole")

# Evaluate the trained model
mean_reward, _ = model.evaluate(env, n_eval_episodes=10)
print("Mean reward:", mean_reward)


In [ ]:
import gym
import tensorflow as tf
from stable_baselines3 import DQN
from stable_baselines3.dqn import MlpPolicy

# Define the CartPole environment
env = gym.make('CartPole-v1')

# Define the neural network architecture for the policy using Keras
def create_model(input_dim, output_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(output_dim, activation='linear')
    ])
    return model

# Create the neural network model
policy_kwargs = dict(
    net_arch=create_model,  # Specify the neural network architecture function
    features_extractor_kwargs={"features_dim": env.observation_space.shape[0]}
)

# Create and train the DQN agent with the specified policy
model = DQN(MlpPolicy, env, policy_kwargs=policy_kwargs, verbose=1)
model.learn(total_timesteps=10000)

# Save the trained model
model.save("dqn_cartpole")

# Evaluate the trained model
mean_reward, _ = model.evaluate(env, n_eval_episodes=10)
print("Mean reward:", mean_reward)
